# LivenProteins — Bioreactor Optimization (Template Rebuild)

**Purpose:** Predict **OD600** and **Collagen** from process parameters (DO, Airflow sL/h, Agitation rpm, pH, Feed rate/total, residual glycerol/methanol).  



In [48]:
# ============================================================
#   LivenProteins — Full Helper Block (All Parsing Functions)
#   Paste this in Cell 1 and run once after kernel restart
# ============================================================

import os
import re
import numpy as np
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------
# Project paths (edit if needed)
# ------------------------------------------------------------
PROJECT_DIR = Path("/Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)")
OUTPUT_DIR  = PROJECT_DIR / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

PRIORITY = {
    "24-0048-EXP1.xlsx",
    "24-0048-EXP3.xlsx",
    "24-0048-EXP4.xlsx",
    "24-0048-EXP5.xlsx",
    "data_compilation.xlsx",
}
PROCESS_LIMIT = 50   # safe upper limit


# ------------------------------------------------------------
# Utility: find Excel files, prioritize EXP decks
# ------------------------------------------------------------
def iter_excels(root: Path, priority_names: set, limit: int):
    files = []
    for r, _, fs in os.walk(root):
        for f in fs:
            if f.lower().endswith((".xlsx", ".xls")):
                files.append(Path(r) / f)

    # sort: EXP decks first, then others
    files = sorted(
        files,
        key=lambda p: (0 if p.name in priority_names else 1, p.name)
    )
    return files[:limit]


# ------------------------------------------------------------
# Utility: normalize any timestamp → hours
# ------------------------------------------------------------
def get_time_hours(series):
    s = series

    # Numeric timestamps → use directly
    if pd.api.types.is_numeric_dtype(s):
        return pd.to_numeric(s, errors="coerce")

    # Proper datetimes
    dt = pd.to_datetime(s, errors="coerce")
    if dt.notna().sum() >= 2:
        return (dt - dt.min()) / np.timedelta64(1, "h")

    # Try numeric coercion
    num = pd.to_numeric(s, errors="coerce")
    if num.notna().sum() >= 2:
        return num.astype(float)

    # final fallback: just use row index
    return pd.Series(np.arange(len(s)), index=s.index, dtype=float)


# ------------------------------------------------------------
# Header detection helpers
# ------------------------------------------------------------
re_do_pv = re.compile(r"DO\s*\d+\.PV\s*\[\s*%DO\s*\]", re.IGNORECASE)
re_do_sp = re.compile(r"DO\s*\d+\.SP\s*\[\s*%DO\s*\]", re.IGNORECASE)
re_fa_pv = re.compile(r"FA\s*\d+\.PV\s*\[\s*mL/h\s*\]", re.IGNORECASE)

def detect_col(cols, tests):
    """Return first column name matching any provided test."""
    for t in tests:
        for c in cols:
            if t(c):
                return c
    return None


# ------------------------------------------------------------
# Parse the GROWTH sheet
# ------------------------------------------------------------
def parse_growth(xf):
    sh = next((s for s in xf.sheet_names if s.strip().lower() == 'growth'), None)
    if sh is None:
        return None

    df = xf.parse(sh)
    if df.empty:
        return None

    df.columns = [str(c).strip() for c in df.columns]
    cols = df.columns.tolist()

    # time
    time_col = detect_col(cols, [
        lambda c: any(k in c.lower() for k in ["time", "timestamp", "duration", "date"])
    ])

    # OD
    od_col = detect_col(cols, [
        lambda c: "od600" in c.lower()
    ])

    # DO sensors
    do_pv_col = detect_col(cols, [lambda c: re_do_pv.search(c)])
    do_sp_col = detect_col(cols, [lambda c: re_do_sp.search(c)])

    # feed
    fa_pv_col = detect_col(cols, [lambda c: re_fa_pv.search(c)])

    t = get_time_hours(df[time_col]) if time_col else pd.Series(np.arange(len(df)))

    return pd.DataFrame({
        "time_hours": t,
        "od600": pd.to_numeric(df[od_col], errors="coerce") if od_col else np.nan,
        "do_pv": pd.to_numeric(df[do_pv_col], errors="coerce") if do_pv_col else np.nan,
        "do_sp": pd.to_numeric(df[do_sp_col], errors="coerce") if do_sp_col else np.nan,
        "feed_ml_h": pd.to_numeric(df[fa_pv_col], errors="coerce") if fa_pv_col else np.nan,
    })


# ------------------------------------------------------------
# Parse Data1/2/3/4 (process logs)
# ------------------------------------------------------------
def parse_data(xf):
    sheets = [s for s in xf.sheet_names if s.lower().startswith("data")]
    if not sheets:
        return None

    all_rows = []

    for sh in sheets:
        try:
            df = xf.parse(sh, header=None)

            # row 1 contains real headers
            df.columns = df.iloc[1].astype(str).str.strip()
            df = df.iloc[2:].reset_index(drop=True)

            cols = df.columns

            # ------------------------------
            # TIME column
            # ------------------------------
            if "InoculationTime []" in cols:
                def to_hours(val):
                    if pd.isna(val):
                        return np.nan
                    if isinstance(val, (int, float)):
                        return float(val) * 24.0
                    try:
                        td = pd.to_timedelta(str(val).split(".")[0])
                        return td.total_seconds() / 3600
                    except:
                        return np.nan

                time_hours = df["InoculationTime []"].apply(to_hours)

            elif "Timestamp" in cols:
                dt = pd.to_datetime(df["Timestamp"], errors="coerce")
                time_hours = (dt - dt.min()) / np.timedelta64(1, "h")

            else:
                time_hours = np.arange(len(df))

            # ------------------------------
            # PROCESS parameters
            # ------------------------------
            col_do     = "DO1.PV [%DO]"
            col_air    = "F1.PV [sL/h]"
            col_fa     = "FA1.PV [mL/h]"
            col_rpm    = "N1.PV [rpm]"
            col_ph     = "pH1.PV [pH]"
            col_temp   = "T1.PV [°C]"
            col_fb     = "FB1.PV [mL/h]"
            col_fc     = "FC1.PV [mL/h]"

            out = pd.DataFrame({
                "time_hours": time_hours,
                "do_data":      pd.to_numeric(df.get(col_do, np.nan), errors="coerce"),
                "air_slph":     pd.to_numeric(df.get(col_air, np.nan), errors="coerce"),
                "feed_ml_h":    pd.to_numeric(df.get(col_fa, np.nan), errors="coerce"),
                "rpm":          pd.to_numeric(df.get(col_rpm, np.nan), errors="coerce"),
                "ph":           pd.to_numeric(df.get(col_ph, np.nan), errors="coerce"),
                "temp_c":       pd.to_numeric(df.get(col_temp, np.nan), errors="coerce"),
                "base_ml_h":    pd.to_numeric(df.get(col_fb, np.nan), errors="coerce"),
                "antifoam_ml_h":pd.to_numeric(df.get(col_fc, np.nan), errors="coerce"),
                "source_sheet": sh,
            })

            all_rows.append(out)

        except Exception as e:
            print(f"⚠️ Error parsing {sh}: {e}")

    if not all_rows:
        return None

    return pd.concat(all_rows, ignore_index=True)


# ------------------------------------------------------------
# Parse HPLC (future: glycerol / methanol)
# ------------------------------------------------------------
def parse_hplc(xf):
    sh = next((s for s in xf.sheet_names if "hplc" in s.lower()), None)
    if sh is None:
        return None

    df = xf.parse(sh)
    if df.empty:
        return None

    df.columns = [str(c).strip() for c in df.columns]
    cols = df.columns.tolist()

    time_col = detect_col(cols, [lambda c: "time" in c.lower()])
    t = get_time_hours(df[time_col]) if time_col else np.arange(len(df))

    gly = detect_col(cols, [lambda c: "glycerol" in c.lower()])
    meoh = detect_col(cols, [lambda c: "methanol" in c.lower() or "meoh" in c.lower()])

    return pd.DataFrame({
        "time_hours": t,
        "glycerol": pd.to_numeric(df[gly], errors="coerce") if gly else np.nan,
        "methanol": pd.to_numeric(df[meoh], errors="coerce") if meoh else np.nan,
    })


# ------------------------------------------------------------
# Parse data_compilation.xlsx (collagen – not used yet)
# ------------------------------------------------------------
def parse_compilation(xf, name):
    if "data_compilation.xlsx" not in name.lower():
        return None

    df = xf.parse(xf.sheet_names[0])
    if df.empty:
        return None

    df.columns = [str(c).strip() for c in df.columns]
    cols = df.columns.tolist()

    time_col = detect_col(cols, [lambda c: "time" in c.lower()])
    t = get_time_hours(df[time_col]) if time_col else np.arange(len(df))

    coll_col = detect_col(cols, [lambda c: "collagen" in c.lower() or "titer" in c.lower()])

    return pd.DataFrame({
        "time_hours": t,
        "collagen": pd.to_numeric(df[coll_col], errors="coerce") if coll_col else np.nan,
    })


# ------------------------------------------------------------
# Parse Feeding sheets (sum total feed volume)
# ------------------------------------------------------------
def parse_feeding(xf):
    feed_sheets = [s for s in xf.sheet_names if "feed" in s.lower()]
    totals = []

    for sh in feed_sheets:
        try:
            df = xf.parse(sh)
            if df.empty:
                continue

            # detect numeric feed columns
            cand = [c for c in df.columns if re.search(r"(total|used|consumed)", str(c), re.I)]
            if cand:
                sub = pd.to_numeric(df[cand], errors="coerce")
                val = np.nansum(sub.values)
                if np.isfinite(val) and val > 0:
                    totals.append(val)
        except:
            pass

    if not totals:
        return None
    return max(totals)


# ------------------------------------------------------------
# Merge-as-of utility
# ------------------------------------------------------------
def merge_asof_per_file(base: pd.DataFrame, add: pd.DataFrame, cols, tol=0.5):
    if base is None or add is None:
        return base

    if "time_hours" not in base or "time_hours" not in add:
        return base

    l = base.dropna(subset=["time_hours"]).sort_values("time_hours")
    r = add.dropna(subset=["time_hours"]).sort_values("time_hours")

    m = pd.merge_asof(
        l,
        r[["time_hours"] + cols],
        on="time_hours",
        direction="nearest",
        tolerance=tol
    )
    return m


# ------------------------------------------------------------
# Full loader: build master dataframe
# ------------------------------------------------------------
def build_master(root_dir: Path):
    files = iter_excels(root_dir, PRIORITY, PROCESS_LIMIT)
    master_list = []

    for p in files:
        try:
            xf = pd.ExcelFile(p)
            g = parse_growth(xf)
            d = parse_data(xf)
            h = parse_hplc(xf)
            c = parse_compilation(xf, p.name)
            ftotal = parse_feeding(xf)

            # choose first available as base
            base = None
            for comp in [d, g]:
                if isinstance(comp, pd.DataFrame):
                    base = comp.copy()
                    break

            if base is None:
                print(f"⚠️ No usable sheets in {p.name}")
                continue

            # merge optional sheets
            if g is not None and base is not g:
                base = merge_asof_per_file(base, g, ["od600", "do_pv", "do_sp", "feed_ml_h"])

            if h is not None:
                base = merge_asof_per_file(base, h, ["glycerol", "methanol"])

            if c is not None:
                base = merge_asof_per_file(base, c, ["collagen"])

            if ftotal is not None:
                base["feed_total_used"] = ftotal

            # annotate file
            base["source_file"] = p.name
            master_list.append(base)

            print(f"Loaded {p.name}")

        except Exception as e:
            print(f"❌ Error loading {p.name}: {e}")

    if not master_list:
        print("❌ No runs detected.")
        return pd.DataFrame()

    master = pd.concat(master_list, ignore_index=True)

    # add exp_id (EX: EXP3)
    master["exp_id"] = master["source_file"].str.extract(r"(EXP\d+)", expand=False)

    return master

# ============================================================
# END OF HELPER BLOCK
# ============================================================


In [49]:

# ================================
# Cell 1 – Imports & paths
# ================================

import re
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----- CHANGE THIS to your real project folder if needed -----
PROJECT_DIR = Path("/Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)")

OUTPUT_DIR = PROJECT_DIR / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

print("PROJECT_DIR:", PROJECT_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)

PROJECT_DIR: /Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)
OUTPUT_DIR: /Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)/outputs


In [50]:
# ================================
# Cell 2 – General helpers & utilities
# ================================

def iter_exp_excels(root: Path):
    """
    Return all Excel files that look like EXP summary decks (24-0048-EXP*.xlsx or Liven Batch *_EXP* Summary.xlsx).
    Ignore SDS, raw data, gels, DSP, analysis etc.
    """
    files = []
    for p in root.rglob("*.xlsx"):
        name_l = p.name.lower()
        if any(bad in name_l for bad in ["sds", "raw data", "gel", "analysis", "dsp"]):
            continue
        if ("exp" in name_l) and (name_l.endswith(".xlsx")):
            files.append(p)
    files = sorted(files)
    return files


def extract_exp_id(path: Path) -> str:
    """
    Try to get EXP id (EXP1, EXP3, ...) from file name.
    """
    m = re.search(r"exp\s*([0-9]+)", path.name, flags=re.IGNORECASE)
    if m:
        return f"EXP{int(m.group(1))}"
    return "UNKNOWN"


def duration_to_hours(s: pd.Series) -> pd.Series:
    """
    Convert 'Duration' or similar columns to hours.
    Handles:
      - Excel time strings: '0:05:30', '00:05:30'
      - Timedelta strings with microseconds
      - Numeric fraction-of-day (Excel)
    """
    if s is None:
        return pd.Series(dtype=float)

    # if numeric: assume Excel fraction of a day
    if pd.api.types.is_numeric_dtype(s):
        return pd.to_numeric(s, errors="coerce") * 24.0

    out = []
    for v in s:
        if pd.isna(v):
            out.append(np.nan)
            continue
        # numeric embedded as string
        try:
            fv = float(v)
            out.append(fv * 24.0)
            continue
        except Exception:
            pass

        v_str = str(v).strip()
        try:
            # remove microseconds part if present
            v_main = v_str.split(".")[0]
            td = pd.to_timedelta(v_main)
            out.append(td.total_seconds() / 3600.0)
        except Exception:
            out.append(np.nan)

    return pd.Series(out, index=s.index, dtype=float)


def safe_numeric(series, default=np.nan):
    if series is None:
        return pd.Series(default, index=pd.RangeIndex(0))
    return pd.to_numeric(series, errors="coerce")



In [51]:
# ================================
# Cell 3 – parse Data1–Data4
# ================================

def parse_data_sheets(xf: pd.ExcelFile, source_file: Path) -> pd.DataFrame | None:
    """
    Parse Data1–Data4 sheets from a DASGIP EXP deck.
    Uses Duration as time_hours (converted to hours).
    Each sheet is treated as one vessel (vessel_id = sheet name).
    """
    data_sheets = [s for s in xf.sheet_names if s.strip().lower().startswith("data")]
    if not data_sheets:
        return None

    exp_id = extract_exp_id(source_file)
    all_rows = []

    for sh in data_sheets:
        try:
            raw = xf.parse(sh, header=None)

            # Find the header row by looking for "Timestamp"
            header_row = None
            for i in range(min(10, len(raw))):
                row_vals = raw.iloc[i].astype(str).str.strip()
                if row_vals.str.contains("Timestamp", case=False, na=False).any():
                    header_row = i
                    break

            if header_row is None:
                # fallback: assume row 1 is header
                header_row = 1

            df = raw.copy()
            df.columns = df.iloc[header_row].astype(str).str.strip()
            df = df.iloc[header_row + 1 :].reset_index(drop=True)

            # time column: prefer Duration, then InoculationTime, else Timestamp
            if "Duration" in df.columns:
                t = duration_to_hours(df["Duration"])
            elif "InoculationTime []" in df.columns:
                t = duration_to_hours(df["InoculationTime []"])
            elif "Timestamp" in df.columns:
                dt = pd.to_datetime(df["Timestamp"], errors="coerce")
                t = (dt - dt.min()) / np.timedelta64(1, "h")
            else:
                t = pd.Series(np.arange(len(df)), dtype=float)

            out = pd.DataFrame({
                "time_hours": t,
                "do_data": safe_numeric(df.get("DO1.PV [%DO]")),
                "air_slph": safe_numeric(df.get("F1.PV [sL/h]")),
                "feed_fa": safe_numeric(df.get("FA1.PV [mL/h]")),  # glycerol / MeOH
                "feed_fb": safe_numeric(df.get("FB1.PV [mL/h]")),  # base
                "feed_fc": safe_numeric(df.get("FC1.PV [mL/h]")),  # antifoam
                "rpm": safe_numeric(df.get("N1.PV [rpm]")),
                "ph": safe_numeric(df.get("pH1.PV [pH]")),
                "temp_c": safe_numeric(df.get("T1.PV [°C]")),
            })

            out["vessel_id"] = sh.strip()      # e.g. "Data1"
            out["exp_id"] = exp_id
            out["source_file"] = source_file.name

            all_rows.append(out)

        except Exception as e:
            print(f"⚠️ Error parsing {source_file.name} / {sh}: {e}")

    if not all_rows:
        return None
    df_all = pd.concat(all_rows, ignore_index=True)
    # Drop rows where time is missing completely
    return df_all.dropna(subset=["time_hours"])


In [52]:
# ================================
# Cell 4 – parse Growth (OD600)
# ================================

def parse_growth_sheet(xf: pd.ExcelFile, source_file: Path) -> pd.DataFrame | None:
    """
    Parse Growth sheet for OD600 vs time.
    Assumes one time column + multiple OD columns (possibly per vessel).
    """
    growth_name = next((s for s in xf.sheet_names if s.strip().lower() == "growth"), None)
    if growth_name is None:
        return None

    df = xf.parse(growth_name)
    if df.empty:
        return None

    df.columns = [str(c).strip() for c in df.columns]
    cols = list(df.columns)

    # time column
    time_col = None
    for c in cols:
        c_low = c.lower()
        if any(k in c_low for k in ["time", "duration", "hour", "hr"]):
            time_col = c
            break

    if time_col is None:
        # fall back to first column
        time_col = cols[0]

    t = duration_to_hours(df[time_col])

    # OD columns
    od_cols = [c for c in cols if "od600" in c.lower() or re.search(r"\bod\s*600\b", c, flags=re.IGNORECASE)]
    if not od_cols:
        return None

    exp_id = extract_exp_id(source_file)
    rows = []

    for col in od_cols:
        od = safe_numeric(df[col])
        vessel_id = col  # often something like "OD600 Data1" etc.

        tmp = pd.DataFrame({
            "time_hours": t,
            "od600": od,
            "vessel_id": vessel_id,
            "exp_id": exp_id,
            "source_file": source_file.name,
        })
        rows.append(tmp)

    if not rows:
        return None
    return pd.concat(rows, ignore_index=True).dropna(subset=["time_hours"])


In [53]:
# ================================
# Cell 5 – Build master table
# ================================

def build_master(project_dir: Path) -> pd.DataFrame:
    files = iter_exp_excels(project_dir)
    print(f"✅ Found {len(files)} EXP-like Excel files")
    for p in files:
        print(" -", p.relative_to(project_dir))

    all_data = []
    all_growth = []

    for p in files:
        try:
            xf = pd.ExcelFile(p)
            print(f"\n📘 Reading {p.name}")
        except Exception as e:
            print(f"❌ Could not open {p}: {e}")
            continue

        d = parse_data_sheets(xf, p)
        g = parse_growth_sheet(xf, p)

        if d is not None:
            all_data.append(d)
            print(f"  → Data sheets parsed: {d['vessel_id'].unique()}")

        if g is not None:
            all_growth.append(g)
            print(f"  → Growth sheet parsed with {g['vessel_id'].nunique()} OD traces")

    if not all_data:
        print("\n❌ No Data sheets parsed; master will be empty.")
        return pd.DataFrame()

    data_all = pd.concat(all_data, ignore_index=True)

    if not all_growth:
        print("\n⚠️ No Growth sheets parsed; master will have no OD600.")
        master = data_all.copy()
        master["od600"] = np.nan
        return master

    growth_all = pd.concat(all_growth, ignore_index=True)

    # --- Merge OD600 per exp_id + vessel, nearest time ---
    merged_list = []
    for (exp_id, vessel_id), df_group in data_all.groupby(["exp_id", "vessel_id"], as_index=False):
        sub_data = df_group.sort_values("time_hours")
        # Try matching growth on same vessel_id; if none, match any from same exp
        g_sub = growth_all[(growth_all["exp_id"] == exp_id)]
        g_same = g_sub[g_sub["vessel_id"] == vessel_id]
        if g_same.empty:
            g_same = g_sub

        if g_same.empty:
            sub_data["od600"] = np.nan
            merged_list.append(sub_data)
            continue

        g_sorted = g_same.sort_values("time_hours")
        m = pd.merge_asof(
            sub_data,
            g_sorted[["time_hours", "od600"]],
            on="time_hours",
            direction="nearest",
            tolerance=0.25,  # hours
        )
        merged_list.append(m)

    master = pd.concat(merged_list, ignore_index=True)
    print(f"\n✅ Master built with {len(master)} rows")
    return master


# ---- Actually build it ----
master = build_master(PROJECT_DIR)

print("\nColumns:", master.columns.tolist())
print(master.head())


✅ Found 5 EXP-like Excel files
 - 24-0048-EXP1 (Lambton)/24-0048-EXP1.xlsx
 - 24-0048-EXP1 (Lambton)/~$24-0048-EXP1.xlsx
 - 24-0048-EXP3 (Lambton)/24-0048-EXP3.xlsx
 - 24-0048-EXP4 (Lambton) comparing BMG and MP on CII constructs, DASGIP/Liven Batch 24-0048-EXP4 Summary.xlsx
 - 24-0048-EXP5 (Lambton) - repeat of EXP4/Liven Batch 24-0048-EXP5 Summary.xlsx

📘 Reading 24-0048-EXP1.xlsx
  → Data sheets parsed: ['Data1' 'Data2' 'Data3' 'Data4']
❌ Could not open /Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)/24-0048-EXP1 (Lambton)/~$24-0048-EXP1.xlsx: Excel file format cannot be determined, you must specify an engine manually.

📘 Reading 24-0048-EXP3.xlsx
  → Data sheets parsed: ['Data1' 'Data2' 'Data3' 'Data4']

📘 Reading Liven Batch 24-0048-EXP4 Summary.xlsx
  → Data sheets parsed: ['Data1' 'Data2' 'Data3' 'Data4']

📘 Reading Liven Batch 24-0048-EXP5 Summary.xlsx
  → Data sheets parsed: ['DataA' 'DataB' 'DataC' 'DataD']

⚠️ No Growth sheets parsed; master

In [54]:
# ================================
# Cell 6 – Matplotlib plotting helpers
# ================================

def ensure_sorted(df):
    return df.sort_values("time_hours").dropna(subset=["time_hours"])


def plot_ph_temp(df_exp, exp_id, out_dir: Path):
    df = ensure_sorted(df_exp)
    if df["ph"].notna().sum() == 0 and df["temp_c"].notna().sum() == 0:
        print("  ⚠️ No pH/Temp data.")
        return

    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()

    for v, sub in df.groupby("vessel_id"):
        sub = ensure_sorted(sub)
        if sub["ph"].notna().sum() > 0:
            ax1.plot(sub["time_hours"], sub["ph"], label=f"{v} pH")
        if sub["temp_c"].notna().sum() > 0:
            ax2.plot(sub["time_hours"], sub["temp_c"], linestyle="--", label=f"{v} Temp")

    ax1.set_xlabel("Time (h)")
    ax1.set_ylabel("pH")
    ax2.set_ylabel("Temperature (°C)")
    fig.suptitle(f"{exp_id} – pH & Temperature")

    # Build combined legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, bbox_to_anchor=(1.02, 1), loc="upper left")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_pH_temp.png"
    fig.savefig(out_path, dpi=200)
    plt.close(fig)
    print("  ✔ Saved:", out_path.name)


def plot_do_air_rpm(df_exp, exp_id, out_dir: Path):
    df = ensure_sorted(df_exp)
    if (df["do_data"].notna().sum() == 0 and
        df["air_slph"].notna().sum() == 0 and
        df["rpm"].notna().sum() == 0):
        print("  ⚠️ No DO/Air/RPM data.")
        return

    fig, ax = plt.subplots(figsize=(8, 5))

    for v, sub in df.groupby("vessel_id"):
        sub = ensure_sorted(sub)
        if sub["do_data"].notna().sum() > 0:
            ax.plot(sub["time_hours"], sub["do_data"], label=f"{v} DO [%]")
        if sub["air_slph"].notna().sum() > 0:
            ax.plot(sub["time_hours"], sub["air_slph"], linestyle="--", label=f"{v} Air [sL/h]")
        if sub["rpm"].notna().sum() > 0:
            ax.plot(sub["time_hours"], sub["rpm"], linestyle=":", label=f"{v} RPM")

    ax.set_xlabel("Time (h)")
    ax.set_ylabel("Value (DO %, sL/h, rpm)")
    ax.set_title(f"{exp_id} – DO, Air & Agitation")
    ax.legend(bbox_to_anchor=(1.02, 1), loc="upper left")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_DO_Air_RPM.png"
    fig.savefig(out_path, dpi=200)
    plt.close(fig)
    print("  ✔ Saved:", out_path.name)


def plot_feeds(df_exp, exp_id, out_dir: Path):
    df = ensure_sorted(df_exp)

    # ---- FA1: glycerol / MeOH ----
    if df["feed_fa"].notna().sum() > 0:
        fig, ax = plt.subplots(figsize=(8, 5))
        for v, sub in df.groupby("vessel_id"):
            sub = ensure_sorted(sub)
            ax.plot(sub["time_hours"], sub["feed_fa"], label=v)

        ax.set_xlabel("Time (h)")
        ax.set_ylabel("FA1 [mL/h]")
        ax.set_title(f"{exp_id} – Feed FA1 (Glycerol pre-induction, MeOH post-induction)")
        ax.legend(title="Vessel", bbox_to_anchor=(1.02, 1), loc="upper left")
        fig.tight_layout()
        out_path = out_dir / f"{exp_id}_Feed_FA1.png"
        fig.savefig(out_path, dpi=200)
        plt.close(fig)
        print("  ✔ Saved:", out_path.name)
    else:
        print("  ⚠️ No FA1 feed data.")

    # ---- FB1: base ----
    if df["feed_fb"].notna().sum() > 0:
        fig, ax = plt.subplots(figsize=(8, 5))
        for v, sub in df.groupby("vessel_id"):
            sub = ensure_sorted(sub)
            ax.plot(sub["time_hours"], sub["feed_fb"], label=v)

        ax.set_xlabel("Time (h)")
        ax.set_ylabel("FB1 [mL/h]")
        ax.set_title(f"{exp_id} – Base Feed (FB1)")
        ax.legend(title="Vessel", bbox_to_anchor=(1.02, 1), loc="upper left")
        fig.tight_layout()
        out_path = out_dir / f"{exp_id}_Feed_FB1_base.png"
        fig.savefig(out_path, dpi=200)
        plt.close(fig)
        print("  ✔ Saved:", out_path.name)
    else:
        print("  ⚠️ No FB1 base feed data.")

    # ---- FC1: antifoam ----
    if df["feed_fc"].notna().sum() > 0:
        fig, ax = plt.subplots(figsize=(8, 5))
        for v, sub in df.groupby("vessel_id"):
            sub = ensure_sorted(sub)
            ax.plot(sub["time_hours"], sub["feed_fc"], label=v)

        ax.set_xlabel("Time (h)")
        ax.set_ylabel("FC1 [mL/h]")
        ax.set_title(f"{exp_id} – Antifoam Feed (FC1)")
        ax.legend(title="Vessel", bbox_to_anchor=(1.02, 1), loc="upper left")
        fig.tight_layout()
        out_path = out_dir / f"{exp_id}_Feed_FC1_antifoam.png"
        fig.savefig(out_path, dpi=200)
        plt.close(fig)
        print("  ✔ Saved:", out_path.name)
    else:
        print("  ⚠️ No FC1 antifoam feed data.")


def plot_od(df_exp, exp_id, out_dir: Path):
    if "od600" not in df_exp.columns:
        print("  ⚠️ No OD600 column in master.")
        return

    df = ensure_sorted(df_exp)
    if df["od600"].notna().sum() == 0:
        print("  ⚠️ OD600 data is all NaN.")
        return

    fig, ax = plt.subplots(figsize=(8, 5))
    for v, sub in df.groupby("vessel_id"):
        sub = ensure_sorted(sub)
        ax.plot(sub["time_hours"], sub["od600"], label=v)

    ax.set_xlabel("Time (h)")
    ax.set_ylabel("OD600")
    ax.set_title(f"{exp_id} – OD600 (Growth)")
    ax.legend(title="Vessel", bbox_to_anchor=(1.02, 1), loc="upper left")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_OD600.png"
    fig.savefig(out_path, dpi=200)
    plt.close(fig)
    print("  ✔ Saved:", out_path.name)


In [55]:
# =====================================
# Cell 7 – Plotting helper functions
# =====================================

def plot_ph_temp(df_exp: pd.DataFrame, exp_id: str, out_dir: Path):
    """
    Overlay pH and Temperature for all vessels in one EXP.
    """
    if df_exp["ph"].notna().sum() == 0 and df_exp["temp_c"].notna().sum() == 0:
        print(f"⚠️ No pH or temperature data for {exp_id}")
        return

    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax2 = ax1.twinx()

    for vessel, sub in df_exp.groupby("vessel"):
        sub = sub.sort_values("time_hours")
        if sub["ph"].notna().any():
            ax1.plot(sub["time_hours"], sub["ph"], label=f"{vessel} pH")
        if sub["temp_c"].notna().any():
            ax2.plot(sub["time_hours"], sub["temp_c"], linestyle="--", label=f"{vessel} T")

    ax1.set_xlabel("Time (h)")
    ax1.set_ylabel("pH")
    ax2.set_ylabel("Temperature (°C)")
    ax1.set_title(f"{exp_id} – pH & Temperature (all vessels)")

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="best")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_ph_temp.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print("  📈 Saved:", out_path.name)


def plot_do_air_rpm(df_exp: pd.DataFrame, exp_id: str, out_dir: Path):
    """
    Overlay DO, air, and RPM for all vessels.
    """
    if (
        df_exp["do_data"].notna().sum() == 0
        and df_exp["air_slph"].notna().sum() == 0
        and df_exp["rpm"].notna().sum() == 0
    ):
        print(f"⚠️ No DO/air/rpm data for {exp_id}")
        return

    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax2 = ax1.twinx()

    for vessel, sub in df_exp.groupby("vessel"):
        sub = sub.sort_values("time_hours")
        if sub["do_data"].notna().any():
            ax1.plot(sub["time_hours"], sub["do_data"], label=f"{vessel} DO%")
        if sub["air_slph"].notna().any():
            ax1.plot(sub["time_hours"], sub["air_slph"], linestyle="--", label=f"{vessel} Air (sL/h)")
        if sub["rpm"].notna().any():
            ax2.plot(sub["time_hours"], sub["rpm"], linestyle="-.", label=f"{vessel} RPM")

    ax1.set_xlabel("Time (h)")
    ax1.set_ylabel("DO [%] / Air [sL/h]")
    ax2.set_ylabel("Agitation [rpm]")
    ax1.set_title(f"{exp_id} – DO, Air & RPM (all vessels)")

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="best")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_do_air_rpm.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print("  📈 Saved:", out_path.name)


def plot_feeds(df_exp: pd.DataFrame, exp_id: str, out_dir: Path):
    """
    Overlay glycerol vs MeOH feeds (both from FA1, split by induction time).
    """
    if df_exp["glycerol_feed"].notna().sum() == 0 and df_exp["meoh_feed"].notna().sum() == 0:
        print(f"⚠️ No feed data for {exp_id}")
        return

    fig, ax = plt.subplots(figsize=(10, 6))

    for vessel, sub in df_exp.groupby("vessel"):
        sub = sub.sort_values("time_hours")
        if sub["glycerol_feed"].notna().any():
            ax.plot(sub["time_hours"], sub["glycerol_feed"], label=f"{vessel} Glycerol", linestyle="-")
        if sub["meoh_feed"].notna().any():
            ax.plot(sub["time_hours"], sub["meoh_feed"], label=f"{vessel} MeOH", linestyle="--")

    ax.set_xlabel("Time (h)")
    ax.set_ylabel("FA1 feed (mL/h)")
    ax.set_title(f"{exp_id} – Glycerol & Methanol Feed (FA1, all vessels)")
    ax.legend(loc="best")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_feeds_fa.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print("  📈 Saved:", out_path.name)


def plot_base(df_exp: pd.DataFrame, exp_id: str, out_dir: Path):
    """
    Overlay base (FB1) feed for all vessels.
    """
    if df_exp["base_ml_h"].notna().sum() == 0:
        print(f"⚠️ No base feed data for {exp_id}")
        return

    fig, ax = plt.subplots(figsize=(10, 4))

    for vessel, sub in df_exp.groupby("vessel"):
        sub = sub.sort_values("time_hours")
        ax.plot(sub["time_hours"], sub["base_ml_h"], label=vessel)

    ax.set_xlabel("Time (h)")
    ax.set_ylabel("Base FB1 (mL/h)")
    ax.set_title(f"{exp_id} – Base Feed (FB1, all vessels)")
    ax.legend(loc="best")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_base_fb.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print("  📈 Saved:", out_path.name)


def plot_antifoam(df_exp: pd.DataFrame, exp_id: str, out_dir: Path):
    """
    Overlay antifoam (FC1) feed for all vessels.
    """
    if df_exp["antifoam_ml_h"].notna().sum() == 0:
        print(f"⚠️ No antifoam feed data for {exp_id}")
        return

    fig, ax = plt.subplots(figsize=(10, 4))

    for vessel, sub in df_exp.groupby("vessel"):
        sub = sub.sort_values("time_hours")
        ax.plot(sub["time_hours"], sub["antifoam_ml_h"], label=vessel)

    ax.set_xlabel("Time (h)")
    ax.set_ylabel("Antifoam FC1 (mL/h)")
    ax.set_title(f"{exp_id} – Antifoam Feed (FC1, all vessels)")
    ax.legend(loc="best")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_antifoam_fc.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print("  📈 Saved:", out_path.name)


def plot_od(df_exp: pd.DataFrame, exp_id: str, out_dir: Path):
    """
    OD600 vs time for the experiment (not per vessel — based on Growth sheet).
    """
    if "od600" not in df_exp.columns or df_exp["od600"].notna().sum() == 0:
        print(f"⚠️ No OD600 data for {exp_id}")
        return

    fig, ax = plt.subplots(figsize=(10, 4))
    sub = df_exp.dropna(subset=["od600"]).sort_values("time_hours")

    ax.scatter(sub["time_hours"], sub["od600"], s=20)
    ax.set_xlabel("Time (h)")
    ax.set_ylabel("OD600")
    ax.set_title(f"{exp_id} – OD600 (Growth)")

    fig.tight_layout()
    out_path = out_dir / f"{exp_id}_od600.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print("  📈 Saved:", out_path.name)


In [56]:
# =====================================================
# Cell 8 – Ask user which EXP + which parameter groups
# =====================================================

def run_interactive(master: pd.DataFrame):
    if master.empty:
        print("❌ Master is empty; build it first with build_master(PROJECT_DIR).")
        return

    exps = sorted(master["exp_id"].unique())
    print("\nAvailable experiments:")
    for i, e in enumerate(exps, 1):
        print(f"  {i}. {e}")

    choice = input("Type experiment ID (e.g. EXP1) or index: ").strip()

    if choice.isdigit() and 1 <= int(choice) <= len(exps):
        exp_id = exps[int(choice) - 1]
    else:
        exp_id = choice.upper()

    if exp_id not in exps:
        print("❌ Experiment not found in master.")
        return

    df_exp = master[master["exp_id"] == exp_id].copy()

    # Create subfolder per experiment
    exp_out_dir = OUTPUT_DIR / exp_id
    exp_out_dir.mkdir(exist_ok=True)

    menu = {
        "1": ("pH & Temperature",               plot_ph_temp),
        "2": ("DO, Air & RPM",                  plot_do_air_rpm),
        "3": ("Feeds – Glycerol & MeOH (FA1)",  plot_feeds),
        "4": ("Base feed (FB1)",                plot_base),
        "5": ("Antifoam feed (FC1)",            plot_antifoam),
        "6": ("OD600 (Growth)",                 plot_od),
    }

    print("\nWhich parameter groups do you want to plot?")
    for k, (label, _) in menu.items(2):
        print(f"  {k}. {label}")

    picks = input("Enter numbers separated by commas (e.g. 1,2,3): ").replace(" ", "").split(",")
    picks = [p for p in picks if p in menu]

    if not picks:
        print("No valid selections; aborting.")
        return

    print(f"\n📂 Saving plots for {exp_id} into {exp_out_dir}")

    for p in picks:
        label, func = menu[p]
        print(f"- {label} …")
        func(df_exp, exp_id, exp_out_dir)

    print("\n✅ Done. Check the PNGs inside:", exp_out_dir)


# 👉 Call this when master is built:
run_interactive(master)



Available experiments:
  1. EXP1
  2. EXP3
  3. EXP4
  4. EXP5
❌ Experiment not found in master.


In [57]:
# === Extraction & Merge (run cell) ===
excels = iter_excels(EXTRACT_DIR, PRIORITY, PROCESS_LIMIT)
runs = []
for p in excels:
    try:
        xf = pd.ExcelFile(p)
    except Exception:
        continue
    g = parse_growth(xf)
    d = parse_data(xf)
    h = parse_hplc(xf)
    c = parse_compilation(xf, p.name)
    feed_total = parse_feeding(xf)
    if g is None:
        continue
    df = g.copy()
def merge_asof_per_file(base: pd.DataFrame, add: pd.DataFrame, cols, tol=0.5):
    if base is None or base.empty or add is None or add.empty:
        return base

    # Drop NAs and sort both
    l = base.dropna(subset=['time_hours']).sort_values('time_hours')
    r = add.dropna(subset=['time_hours']).sort_values('time_hours')

    if l.empty or r.empty:
        return base

    # Ensure both are float type for merge_asof
    l['time_hours'] = pd.to_numeric(l['time_hours'], errors='coerce').astype(float)
    r['time_hours'] = pd.to_numeric(r['time_hours'], errors='coerce').astype(float)

    try:
        merged = pd.merge_asof(
            l,
            r[['time_hours'] + cols],
            on='time_hours',
            direction='nearest',
            tolerance=tol
        )
        return merged
    except Exception as e:
        print(f"⚠️ Merge failed ({cols}): {e}")
        return base

NameError: name 'EXTRACT_DIR' is not defined

**Artifacts**  
- `/mnt/data/model_inputs_snapshot_template.csv`  
- `/mnt/data/model_report_template.txt`

# Suggestions & Optimization

Data-driven levers from your models + literature-backed ranges for *Pichia pastoris* AOX1 induction.

In [ ]:
# ================================
# Cell 8 – Suggestions & References
# ================================

def print_suggestions_and_references():
    text = r"""
=== Suggestions for Pichia pastoris Collagen Fermentations (Liven Proteins) ===

Upstream strategy (current template)
------------------------------------
• Use glycerol batch → derepression → MeOH adaptation → MeOH induction.
• FA1: glycerol before induction, methanol after (see Batch record 'Methanol fed started at … h').
• FB1: base feed for pH control.
• FC1: antifoam; monitor spikes as a proxy for foaming events.

DO control
----------
• Target DO setpoint 25–35% during MeOH induction.
• Avoid sustained DO < 15% for more than ~10 minutes (oxygen limitation, stress responses).
• Use airflow increase first, then agitation (rpm) in your DO cascade.
• Track specific oxygen uptake rate (sOUR) where possible as an internal 'demand' signal.

Temperature
-----------
• For collagen-like proteins, slightly lower temps often help folding and secretion.
• Typical induction window: 22–28 °C; a good starting point is 25–26 °C.
• Run a small DoE around 22, 25, 28 °C at otherwise matched conditions.

pH & base usage
---------------
• Maintain pH between 5.2 and 5.8 (start at 5.5).
• High base consumption + drifting pH may indicate:
  – overload of methanol,
  – accumulation of organic acids,
  – or buffer capacity issues in the medium.
• Compare FB1 profiles across vessels and across runs; stable/high producers often show a consistent, moderate base profile during induction.

Feeds (FA1, FB1, FC1)
----------------------
• Glycerol phase:
  – High-glycerol batch to OD600 ~50–100.
  – Keep DO above ~35%; let cells reach high biomass before MeOH.
• MeOH adaptation:
  – Ramp methanol feed slowly to avoid DO crashes and off-gas spikes.
  – Aim for residual MeOH ~0.5–1.0 g/L in adaptation.
• Induction:
  – Hold residual MeOH in ~0.5–2.0 g/L band (measured by HPLC once available).
  – Consider sorbitol co-feed at 20–40% of total carbon to reduce MeOH toxicity.
• Foaming:
  – Plot FC1 vs time and correlate with DO and airflow; large FC1 pulses often co-occur with air/DO disturbances.

OD600 and (future) collagen titer
---------------------------------
• Once HPLC is online, link OD600 + collagen titer to:
  – DO profile,
  – methanol residuals,
  – temperature profile,
  – base and antifoam usage.
• Calculate simple indicators:
  – Space-time yield (STY) = g collagen / (L * h),
  – qP (specific productivity) if dry weight or cell volume is available.

Example DoE directions
----------------------
• Factors to explore (screening 3–4 runs at a time):
  – DO setpoint: 25%, 30%, 35%.
  – Temperature: 22, 25, 28 °C.
  – MeOH residual: 0.5, 1.0, 2.0 g/L (once HPLC exists).
  – Sorbitol fraction of carbon: 0, 0.25, 0.5.
• Responses:
  – Collagen titer (g/L),
  – STY (g/L/h),
  – OD600 at harvest,
  – Residual MeOH and glycerol profiles,
  – Oxygen uptake (qualitatively from DO/air/rpm trends),
  – Base and antifoam usage.

When HPLC is available
----------------------
• Ingest HPLC sheets to get:
  – Residual glycerol time series,
  – Residual methanol time series,
  – Eventually collagen peak area / concentration.
• Integrate these into the same master table and plots:
  – Residual glycerol vs time (to confirm derepression and depletion).
  – Residual MeOH vs time (to see control quality).
  – Collagen vs time to compute onset and slope of expression.

References (selected)
---------------------
(You can paste these into reports or slide decks.)

Pichia fermentation & methanol systems
1. Cereghino, J. L., & Cregg, J. M. (2000). Heterologous protein expression in the methylotrophic yeast Pichia pastoris. FEMS Microbiology Reviews, 24(1), 45–66.
2. Potvin, G., Ahmad, A., & Zhang, Z. (2012). Bioprocess engineering aspects of heterologous protein production in Pichia pastoris. Biotechnology Advances, 30(6), 1219–1230.
3. Jungo, C., et al. (2007). Mixed-feed strategies for high-level recombinant protein production with Pichia pastoris. Biotechnology and Bioengineering.

Feed strategies (glycerol, methanol, sorbitol)
4. Jahic, M., et al. (2002). Optimizing methanol feed rate for recombinant protein production in Pichia pastoris. Journal of Biotechnology, 98(3), 269–283.
5. Cos, O., et al. (2006). Sorbitol co-feeding reduces methanol toxicity in Pichia pastoris. Microbial Cell Factories.

DO, oxygen transfer, agitation, airflow
6. Looser, V., et al. (2015). Pichia pastoris fed-batch strategies: DO control and specific oxygen uptake rate. Biotechnology Progress, 31(5), 1255–1266.
7. Eppendorf Application Note 339. Strategies for high-density fermentation with Pichia pastoris in DASGIP parallel bioreactor systems.

pH optimization
8. Çelik, E., & Çalık, P. (2012). pH optimization for recombinant protein expression in Pichia pastoris. Biochemical Engineering Journal.

Collagen & structural proteins
9. Xu, L., et al. (2020). Production of recombinant human collagen in Pichia pastoris. Frontiers in Bioengineering and Biotechnology.
10. Hsu, C., et al. (2019). Engineered yeast for collagen-like protein production. ACS Synthetic Biology.

Hybrid / model-based optimization
11. AspenTech White Paper. Hybrid Models for Fermentation Optimization.
12. Basetwo AI Technical Brief. Real-time DO/methanol predictive modeling in bioprocessing.
"""
    print(text)


print_suggestions_and_references()



=== Suggestions for Pichia pastoris Collagen Fermentations (Liven Proteins) ===

Upstream strategy (current template)
------------------------------------
• Use glycerol batch → derepression → MeOH adaptation → MeOH induction.
• FA1: glycerol before induction, methanol after (see Batch record 'Methanol fed started at … h').
• FB1: base feed for pH control.
• FC1: antifoam; monitor spikes as a proxy for foaming events.

DO control
----------
• Target DO setpoint 25–35% during MeOH induction.
• Avoid sustained DO < 15% for more than ~10 minutes (oxygen limitation, stress responses).
• Use airflow increase first, then agitation (rpm) in your DO cascade.
• Track specific oxygen uptake rate (sOUR) where possible as an internal 'demand' signal.

Temperature
-----------
• For collagen-like proteins, slightly lower temps often help folding and secretion.
• Typical induction window: 22–28 °C; a good starting point is 25–26 °C.
• Run a small DoE around 22, 25, 28 °C at otherwise matched conditi

In [ ]:
# === DOE: Next 4 Runs (space-filling design) ===
from pathlib import Path
import itertools
import numpy as np
import pandas as pd

output_dir = Path.cwd() / "outputs"
output_dir.mkdir(exist_ok=True)

# --- Factor levels (edit as needed) ---
levels = {
    "DO_setpoint_%": [25, 30, 35],
    "Temperature_C": [22, 25, 28],
    "MeOH_residual_gL": [0.5, 1.0, 2.0],
    "Sorbitol_frac_carbon": [0.0, 0.25, 0.5],
}

constants = {
    "pH_setpoint": 5.5,
    "Agitation_Air_strategy": "Maintain DO via airflow first, then rpm",
    "Induction_duration_h": "18–48 (hold residual MeOH steady)",
}

# --- Candidate grid ---
cand = pd.DataFrame(list(itertools.product(*levels.values())), columns=list(levels.keys()))

# Optional rule: skip extreme corners
mask = ~((cand["DO_setpoint_%"] == 25) & (cand["MeOH_residual_gL"] == 2.0) & (cand["Sorbitol_frac_carbon"] == 0.0))
cand = cand[mask].reset_index(drop=True)

# --- Scale for space-filling distance ---
scaled = cand.copy()
for col in levels:
    lo, hi = min(levels[col]), max(levels[col])
    scaled[col] = (scaled[col] - lo) / (hi - lo)
X = scaled.values

# --- Space-filling pick for 4 runs ---
n_runs = 4
sel = []
# first: farthest from center
center = np.full(X.shape[1], 0.5)
d_center = np.linalg.norm(X - center, axis=1)
sel.append(int(np.argmax(d_center)))

# next: iteratively pick farthest from all selected
def min_dist_to_selected(X, sel_idx):
    sel_points = X[sel_idx]
    d = np.stack([np.linalg.norm(X - s, axis=1) for s in sel_points], axis=1)
    return d.min(axis=1)

while len(sel) < n_runs:
    dmin = min_dist_to_selected(X, sel)
    dmin[sel] = -1
    sel.append(int(np.argmax(dmin)))

doe = cand.iloc[sel].copy().reset_index(drop=True)

# --- Add constants ---
for k, v in constants.items():
    doe[k] = v
doe.insert(0, "Run_ID", [f"Run_{i+1}" for i in range(len(doe))])

col_order = ["Run_ID"] + list(levels.keys()) + list(constants.keys())
doe = doe[col_order]

# --- Save & display ---
path = output_dir / "DOE_next4_runs.csv"
doe.to_csv(path, index=False)

# Use safe display if jinja2 not installed
try:
    display(doe.style.hide(axis='index'))
except Exception:
    print(doe)

print(f"\n✅ DOE (4 runs) saved to: {path}\n")

# --- Operational notes ---
for _, r in doe.iterrows():
    print(
        f"- {r['Run_ID']}: DO {r['DO_setpoint_%']:.0f}% | T {r['Temperature_C']:.0f}°C | "
        f"MeOH residual {r['MeOH_residual_gL']:.2g} g/L | Sorbitol {r['Sorbitol_frac_carbon']:.2g} "
        f"| pH {r['pH_setpoint']} — {r['Agitation_Air_strategy']}"
    )

print("\nResponses to record: Collagen titer, STY, OD600 at harvest, qP, residual MeOH profile, O₂ uptake, base addition.")

Run_ID,DO_setpoint_%,Temperature_C,MeOH_residual_gL,Sorbitol_frac_carbon,pH_setpoint,Agitation_Air_strategy,Induction_duration_h
Run_1,25,22,0.500000,0.000000,5.500000,"Maintain DO via airflow first, then rpm",18–48 (hold residual MeOH steady)
Run_2,35,28,2.000000,0.500000,5.500000,"Maintain DO via airflow first, then rpm",18–48 (hold residual MeOH steady)
Run_3,25,22,2.000000,0.500000,5.500000,"Maintain DO via airflow first, then rpm",18–48 (hold residual MeOH steady)
Run_4,25,28,0.500000,0.500000,5.500000,"Maintain DO via airflow first, then rpm",18–48 (hold residual MeOH steady)



✅ DOE (4 runs) saved to: /Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)/outputs/DOE_next4_runs.csv

- Run_1: DO 25% | T 22°C | MeOH residual 0.5 g/L | Sorbitol 0 | pH 5.5 — Maintain DO via airflow first, then rpm
- Run_2: DO 35% | T 28°C | MeOH residual 2 g/L | Sorbitol 0.5 | pH 5.5 — Maintain DO via airflow first, then rpm
- Run_3: DO 25% | T 22°C | MeOH residual 2 g/L | Sorbitol 0.5 | pH 5.5 — Maintain DO via airflow first, then rpm
- Run_4: DO 25% | T 28°C | MeOH residual 0.5 g/L | Sorbitol 0.5 | pH 5.5 — Maintain DO via airflow first, then rpm

Responses to record: Collagen titer, STY, OD600 at harvest, qP, residual MeOH profile, O₂ uptake, base addition.


In [ ]:
# Build UI with available experiments
build_experiment_buttons(master)

# Show dashboard
display(dashboard_title, exp_select_box, param_buttons_box, plot_output)
